In [ ]:
def get_props(dataZmax, mask):
    props=skimage.measure.regionprops_table(mask, dataZmax, properties=('label', 'area', 'intensity_mean', 'intensity_std'),)
    df=pd.DataFrame(props)
    mean_columns=[column for column in df.columns.values if "intensity_mean" in column]
    df["pooled norm mean"]=(df[mean_columns]/df[mean_columns].loc[0]).mean(axis=1)

    return df

In [1]:
def mask_sliceedge(data):
    datasmooth = skimage.filters.gaussian(data, sigma=50)
    cx, cy = np.shape(datasmooth)
    cx = np.round(cx/2).astype(int)
    cy = np.round(cy/2).astype(int)
    window = 2000
    ifmean = np.mean(datasmooth[cx-window:cx+window, cy-window:cy+window])
    ifstd = np.std(datasmooth[cx-window:cx+window, cy-window:cy+window])
    datasmoothz = (datasmooth - ifmean)/ifstd
    mask = np.ones_like(datasmoothz)*(datasmoothz>5)
    mask[cx-window:cx+window, cy-window:cy+window] = 0
    return mask

In [2]:
def mask_background(data):
    datasmooth = skimage.filters.gaussian(data, sigma=50)
    cx, cy = np.shape(datasmooth)
    cx = np.round(cx/2).astype(int)
    cy = np.round(cy/2).astype(int)
    window = 2000
    ifmean = np.mean(datasmooth[cx-window:cx+window, cy-window:cy+window])
    ifstd = np.std(datasmooth[cx-window:cx+window, cy-window:cy+window])
    datasmoothz = (datasmooth - ifmean)/ifstd
    mask = np.ones_like(datasmoothz)*(datasmoothz<-3)
    return mask

In [ ]:
def get_means_new(metadata, rdr, r_contour, limit):
    tifroot="/Volumes/LaCie/data/rEO_08/histology/20x-tiffs"
    # zoffset=2
    control_locations = [1,2,3,4]
    pooled_means=[]
    normed_means=np.array([])
    allmeans=[]
    for filename in metadatadf['filename']:
        exclude = metadatadf['exclude'].loc[metadatadf['filename']==filename].values[0]
        if not exclude:
            print(filename)
            cx, cy, v_x, v_y, v_r, r_base, z_start, z_stop, plane_selected, control_location = parse_metadata(metadata)
            
            z_start=8
            z_stop=12
            dataAll={}
            for c in range(3):
                planes=[]
                for z in range(z_start, z_stop):
                    plane = rdr.read(series=series, c=c, z=z, t=t, rescale=False)
                    # ensure a plain contiguous ndarray
                    planes.append(np.ascontiguousarray(plane))
                planesMaxProj = np.max(planes, axis=2)
                x,y = np.shape(planesMaxProj)
                dataAll[f"C_{c}"] = planesMaxProj
                num_regions, mask=gen_contours(planesMaxProj, [np.round(x/2).astype(int), np.round(y/2).astype(int)], 
                                               10, 40, limit_um=320)
            
            
            # for control_location in range(1,5):                
            num_regions, mask=gen_contours(data, [cx, cy], r_base, r_contour, limit_um=limit)
            mask=gen_control_area(mask, r_base, r_contour, limit_um=limit, control_location=control_location)
            mask = set_reference(mask)
            
            mask[mask==65535] = 0
            zmid = np.round((z_start+z_stop)/2).astype(int)
            if v_r:
                mask=mask_vessels(mask, v_x, v_y, v_r)
            mask = mask_stitches(mask)
            fig, (ax1, ax2) = plt.subplots(1, 2)
            ax1.imshow(dataMaxProj, cmap='gray')
            ax2.imshow(mask, cmap='jet')
            plt.show()
            
            
            
                
            props=skimage.measure.regionprops_table(mask, dataMaxProj, properties=('label', 'area', 'intensity_mean', 'intensity_std'),)
            df=pd.DataFrame(props)
            mean_columns=[column for column in df.columns.values if "intensity_mean" in column]
            df["pooled norm mean"]=(df[mean_columns]/df[mean_columns].loc[0]).mean(axis=1)
            # allmeans.append((df[mean_columns]/df[mean_columns].loc[0]).values)
            print(df)
            pooled_means.append(df["pooled norm mean"].values[:2*num_regions+1])
            print(np.mean(df["pooled norm mean"].values[num_regions+1:2*num_regions+1]))    
    pooled_means=np.array(pooled_means)

    return pooled_means, allmeans

In [ ]:
def get_means(metadatadf, staining, r_contour, limit):
    tifroot="/Volumes/LaCie/data/rEO_08/histology/20x-tiffs"
    # zoffset=2
    control_locations = [1,2,3,4]
    pooled_means=[]
    normed_means=np.array([])
    allmeans=[]
    for filename in metadatadf['filename']:
        exclude = metadatadf['exclude'].loc[metadatadf['filename']==filename].values[0]
        if not exclude:
            print(filename)
            cx = metadatadf['center_x'].loc[metadatadf['filename']==filename].values[0]
            cy = metadatadf['center_y'].loc[metadatadf['filename']==filename].values[0]
            v_x = metadatadf['v_x'].loc[metadatadf['filename']==filename].values[0]
            v_y = metadatadf['v_y'].loc[metadatadf['filename']==filename].values[0]
            v_r = np.round(metadatadf['v_r'].loc[metadatadf['filename']==filename].values[0]/0.325*1.5).astype(int)
            
            r_base = metadatadf['r_base'].loc[metadatadf['filename']==filename].values[0]
            r_base = r_base + 10
            
            z_start = metadatadf['plane_start'].loc[metadatadf['filename']==filename].values[0]
            z_stop = metadatadf['plane_stop'].loc[metadatadf['filename']==filename].values[0]
            plane_selected = metadatadf['plane_selected'].loc[metadatadf['filename']==filename].values[0]
            
            control_location = metadatadf['control_location'].loc[metadatadf['filename']==filename].values[0]
            filename=filename+"_"+staining+".tif"
            data = imread(os.path.join(tifroot, filename))
            data=np.moveaxis(data, 0, -1)
            if not plane_selected:
                z_start=9
                z_stop=12
                dataMaxProj = np.max(data[:,:,z_start:z_stop], axis=2)
            else:
                dataMaxProj = np.max(data[:,:,plane_selected:plane_selected+3], axis=2)
            
            # for control_location in range(1,5):                
            num_regions, mask=gen_contours(data, [cx, cy], r_base, r_contour, limit_um=limit)
            mask=gen_control_area(mask, r_base, r_contour, limit_um=limit, control_location=control_location)
            mask = set_reference(mask)
            
            mask[mask==65535] = 0
            zmid = np.round((z_start+z_stop)/2).astype(int)
            if v_r:
                mask=mask_vessels(mask, v_x, v_y, v_r)
            mask = mask_stitches(mask)
            fig, (ax1, ax2) = plt.subplots(1, 2)
            ax1.imshow(dataMaxProj, cmap='gray')
            ax2.imshow(mask, cmap='jet')
            plt.show()
            
            
            
                
            props=skimage.measure.regionprops_table(mask, dataMaxProj, properties=('label', 'area', 'intensity_mean', 'intensity_std'),)
            df=pd.DataFrame(props)
            mean_columns=[column for column in df.columns.values if "intensity_mean" in column]
            df["pooled norm mean"]=(df[mean_columns]/df[mean_columns].loc[0]).mean(axis=1)
            # allmeans.append((df[mean_columns]/df[mean_columns].loc[0]).values)
            print(df)
            pooled_means.append(df["pooled norm mean"].values[:2*num_regions+1])
            print(np.mean(df["pooled norm mean"].values[num_regions+1:2*num_regions+1]))    
    pooled_means=np.array(pooled_means)

    return pooled_means, allmeans